<a href="https://colab.research.google.com/github/spectraldoy/music-transformer/blob/main/Generate_Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### © Copyright 2021 Aditya Gomatam

This file is part of music-transformer (https://github.com/spectraldoy/music-transformer), my project to build and
train a Music Transformer. music-transformer is open-source software licensed under the terms of the GNU General
Public License v3.0. music-transformer is free software: you can redistribute it and/or modify it under the terms of
the GNU General Public License as published by the Free Software Foundation, either version 3 of the License,
or (at your option) any later version. music-transformer is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
See the GNU General Public License for more details. A copy of this license can be found within the GitHub repository
for music-transformer, or at https://www.gnu.org/licenses/gpl-3.0.html.

# Generate Music with Music Transformer!

If you're seeing this on the GitHub repository:

1. Click the Open in Colab link
2. The notebook should be pre-configured with a GPU, but in case it isn't, go to Runtime > Change runtime type, and select GPU for Hardware accelerator
3. Run the cells you wish to run with the Play buttons at their top lefts

This Notebook lets you play with pretrained [Music Transformer](https://arxiv.org/pdf/1809.04281.pdf) models to generate Western Classical piano music.

There are 3 models available on the [GitHub repository](https://github.com/spectraldoy/music-transformer), of which the one trained on pieces by Chopin is automatically loaded, and with which you can immediately generate music. If you wish, you may edit the code under the markdown cells in this notebook to generate music with another of those models, or even upload your own to generate music with. I find that all of my models are highly prone to repeated notes and wandering melodies, but their harmonies are amazingly consistent.

In [ ]:
#@title Set up Generation
#@markdown Clone into the GitHub repository, import required libraries,
#@markdown download the Google Magenta SoundFont, and set up variables
#@markdown for music generation. You must run this cell if you want to generate any music..
%%capture

import os
from google.colab import files
from random import randint
from IPython.display import Audio

!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 ./ 

!apt install fluidsynth
!pip install mido

!git clone https://github.com/spectraldoy/music-transformer

gen_path = "./gen_audio.mid"
wav_path = "./gen_audio.wav"
model_path = "./music-transformer/models/chopintransformerv5.pt"

In [ ]:
#@title Generate a Piano Performance from Scratch!

#@markdown The Music Transformer
#@markdown is an autoregressive model, which means it generates outputs one
#@markdown at a time, then looks to all its previous outputs in order to
#@markdown generate the rest. This process should take about a minute or so on a GPU in the best case.
#@markdown Sometimes these models just don't stop generating, which is why you can also KeyboardInterrupt,
#@markdown or press the stop button at the top left to interrupt execution and save whatever has already been
#@markdown generated.
#@markdown
#@markdown Additionally, you can set the approximate tempo here, but the actual tempo
#@markdown of the output will depend greatly on what notes and what rhythm
#@markdown are generated by the model.

tempo = 115 #@param {type:'slider', min:32, max:208}

#@markdown Note that 
#@markdown the model cannot generate complete pieces, but only
#@markdown abrupt "sections" of pieces. This is because only
#@markdown about 2000 MIDI events could be input at a time
#@markdown during training, whereas most pieces consist of 10 000 - 100 000 MIDI events. 

choice = randint(0, 1)
temp = choice * randint(500, 800) / 1000 + (1 - choice) * randint(1000, 1200) / 1000
!python ./music-transformer/generate.py {model_path} {gen_path} -v -t {temp} -tm {tempo}

print("Creating playable audio...")
os.system(f"fluidsynth -ni Yamaha-C5-Salamander-JNv5.1.sf2 {gen_path} -F {wav_path} -r 44100 -g 1.0")
Audio(wav_path)

In [ ]:
#@title Download Performance as MIDI file
#@markdown You can download the generated .wav file by
#@markdown clicking on the 3 dots in the displayed 
#@markdown Audio. Run this cell to download 
#@markdown the performance as MIDI.
files.download(gen_path)